# **Question 4**

## Importing Library and Data Loading

In [ ]:
import pandas as pd
data = pd.read_excel("/content/Question-4.xlsx", skiprows=8, header=0)
data.rename(columns = {"Unnamed: 0" : "Date"}, inplace = True)


**A) i. Computing Standard Deviation, Value at Risk and Conditional Value at Risk between NVDA and NVDL during January 2024 - August 2024**

In [ ]:
import numpy as np
confidence_level = 0.95
var_percentile = 1-confidence_level

def risk_metrics(stock):
  std_dev = stock.std()
  var_95 = np.percentile(stock, var_percentile*100)
  cvar_95 = stock[stock <= var_95].mean()
  return std_dev, var_95, cvar_95

std_NVDA, var_NVDA, cvar_NVDA = risk_metrics(data["NVDA"])
std_NVDL, var_NVDL, cvar_NVDL = risk_metrics(data["NVDL"])

print(f"NVDA - Standard Deviation: {round(std_NVDA*100,4)}% ; VaR 95%: {round(var_NVDA*100,4)}% ; CVaR 95%: {round(cvar_NVDA*100,4)}%")
print("\n")
print(f"NVDL - Standard Deviation: {round(std_NVDL*100,4)}% ; VaR 95%: {round(var_NVDL*100,4)}% ; CVaR 95%: {round(cvar_NVDL*100,4)}%")

NVDA - Standard Deviation: 3.746% ; VaR 95%: -6.121% ; CVaR 95%: -7.0725%


NVDL - Standard Deviation: 7.4795% ; VaR 95%: -12.0288% ; CVaR 95%: -14.1617%


**B) i. Regression estimates of daily returns of NVDL on NVDA**

In [ ]:
# Running a regression
from statsmodels.formula.api import ols
import statsmodels.api as sm

X = sm.add_constant(data["NVDA"])
Y = data["NVDL"]
model = sm.OLS(data["NVDL"], X).fit()
beta_0, beta_1 = model.params  # Intercept and slope
conf_int = model.conf_int()  # Confidence intervals for coefficients

print(f"Intercept (beta_0): {round(beta_0,6)}, Slope (beta_1): {round(beta_1,6)}")
print("Confidence Intervals:\n", conf_int)

Intercept (beta_0): -0.000612, Slope (beta_1): 1.995975
Confidence Intervals:
               0         1
const -0.000935 -0.000289
NVDA   1.987407  2.004543


**C) i. Estimating the annual performance drag**

In [ ]:
# Performance Drag
annualized_drag = round(beta_0 * 252,6)  # Annualizing the daily intercept
print(f"Estimated Annual Performance Drag:{annualized_drag}%")

Estimated Annual Performance Drag:-0.154134%


**D) i. Monthly and Total Return Comparison between NVDA and NVDL**

In [ ]:
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)

monthly_nvda_returns = round(((1 + data['NVDA']).resample('ME').prod() - 1)*100,4)
monthly_nvdL_returns = round(((1 + data['NVDL']).resample('ME').prod() - 1)*100,4)

# Calculate overall period cumulative return (Feb - Aug)
total_return_nvda = (1 + data['NVDA']).prod() - 1
total_return_nvdL = (1 + data['NVDL']).prod() - 1

print(f"Monthly NVDA Returns %:\n {round(monthly_nvda_returns,4)}%")
print("\n")
print(f"Monthly NVDL Returns %:\n {round(monthly_nvdL_returns,4)}%")
print("\n")
print(f"Total Period Return NVDA: {round(total_return_nvda*100,4)}%")
print("\n")
print(f"Total Period Return NVDL: {round(total_return_nvdL*100,4)}%")


Monthly NVDA Returns %:
 Date
2024-02-29    28.5970
2024-03-31    14.2099
2024-04-30    -4.3724
2024-05-31    26.8781
2024-06-30    12.6996
2024-07-31    -5.2781
2024-08-31     2.0084
Freq: ME, Name: NVDA, dtype: float64%


Monthly NVDL Returns %:
 Date
2024-02-29    59.1128
2024-03-31    25.7576
2024-04-30   -12.2169
2024-05-31    55.7233
2024-06-30    22.5983
2024-07-31   -14.6801
2024-08-31    -1.6009
Freq: ME, Name: NVDL, dtype: float64%


Total Period Return NVDA: 94.0497%


Total Period Return NVDL: 181.5333%


**E) i. Simulations of stock 10% up on Day 1 and 9% down on Day 2**

In [ ]:
def two_day_return(initial_return, final_return, leverage=1):
    day1 = 1 + leverage * initial_return
    day2 = 1 + leverage * final_return
    return day1 * day2 - 1

nvda_two_day_return = two_day_return(0.10, -0.09)
nvdL_two_day_return = two_day_return(0.10, -0.09, leverage=2)

print("\nTwo-Day Return Scenario:")
print(f"Two-day return NVDA: {nvda_two_day_return*100}%")
print(f"Two-day return NVDL: {nvdL_two_day_return*100}%")


Two-Day Return Scenario:
Two-day return NVDA: 0.10000000000001119%
Two-day return NVDL: -1.6000000000000014%


**F) i. Event Simulations**

In [ ]:
print("\n4F: Investment Choice Explanation")
print("For a bullish short-term trader, NVDL (the leveraged ETF) may be preferred due to its amplified returns as it is leveraged.")
print("For a long-term investor, NVDA stock is likely better, as NVDL's leverage and rebalancing costs can erode returns over time.")


4F: Investment Choice Explanation
For a bullish short-term trader, NVDL (the leveraged ETF) may be preferred due to its amplified returns as it is leveraged.
For a long-term investor, NVDA stock is likely better, as NVDL's leverage and rebalancing costs can erode returns over time.
